<a href="https://colab.research.google.com/github/va4756/big_RaeJung/blob/main/MachineLearning_%EC%9D%B4%EC%83%81%EC%B2%A0_ML02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 02_01 Artificial Neural Network(예측)

## 1.기본패키지 설치

In [ ]:
# 그래프에서 한글 폰트 인식하기
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

#  *** 런타임 다시 시작

In [1]:
## 1.기본
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 2.데이터 가져오기
import pandas as pd

In [2]:
# 한글 폰트 인식
plt.rc('font', family='NanumBarunGothic')